In [1]:
import pandas as pd
import datetime
from transformers import pipeline
import json

/Users/thibaultrolland/anaconda3/envs/arcomenv/lib/python3.8/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_12390/777748067.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")


In [3]:
def clean_themes(themes):
    if type(themes) == str:
        themes = themes.split(',')
        themes = [theme.strip() for theme in themes]
        themes = [theme.lower() for theme in themes]
        return themes
    return []

arcom_df['clean_theme'] = arcom_df['Thème'].apply(clean_themes)
arcom_df['clean_date'] = [datetime.datetime.strptime(x, '%d/%m/%Y').date() for x in arcom_df['Date']]
arcom_df['visual_and_script'] = arcom_df['Visuel'] + arcom_df['Script']

In [4]:
food_sectors = ['AALIMENTATION', 'BOISSONS', 'TOURISME-RESTAURATION']
food_df = arcom_df[[sector in food_sectors for sector in arcom_df['Secteur']]]
food_df = food_df[~food_df['Script'].isna()]
food_df = food_df[~food_df['Visuel'].isna()]
food_df['original_index'] = food_df.index
food_df = food_df.reset_index()

In [5]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")
hypothesis_template = "Voici une description et un visuel pour une publicité, un de ses thèmes est : {}"

In [6]:
themes = [
    'gourmandise', 
    'santé', 
    'nutrition', 
    'joie', 
    'goût', 
    'humour', 
    'partage', 
    'authenticité', 
    'tradition', 
    'qualité', 
    'innovation', 
    'ecoresponsabilité', 
    'bien-être', 
    'famille', 
    'fête', 
    'simplicité', 
    'praticité', 
    'histoire', 
    'évasion', 
    'énergie', 
    'saisonnalité'
    'plaisir coupable'
    ]
#TODO: try with emotions


In [7]:
sample_len = 2000
sample = food_df.sample(sample_len)

In [8]:
output_list = []


i = 0
for index, row in sample.iterrows():
    print(f'computing zero shot for ad : {i+1} / {sample_len}', end='\r')
    original_index = row['original_index']
    try:
        output= zeroshot_classifier(row['visual_and_script'], themes, hypothesis_template=hypothesis_template, multi_label=True)
        output['error'] = False
    except KeyboardInterrupt:
        print('Interrupted by user.')
    except:
        print(f'{i+1} / {sample_len}; error : couldn\'t proceed id {original_index}')
        output = {'error': True}
    output['id'] = original_index
    output_list.append(output)
    i += 1



1343 / 2000; error : couldn't proceed id 31879
1347 / 2000; error : couldn't proceed id 27596
1349 / 2000; error : couldn't proceed id 28565
1356 / 2000; error : couldn't proceed id 14911
1392 / 2000; error : couldn't proceed id 33007


In [9]:
with open('11a_zeroshot_food.json', 'w') as json_file:
    json.dump(output_list, json_file)